In [ ]:
DATA_PREPARATION_VOTE_METHOD = "sum_and_normalize" 

import os
import pandas as pd
import numpy as np
import torch
import torch.nn.functional as F
from torch.utils.data import  DataLoader
from tqdm import tqdm
import sys
    
if bool(os.environ.get("KAGGLE_URL_BASE", "")):
    sys.path.insert(0, "/kaggle/input/hsm-source-files")
    
    tcn_source_path = '/kaggle/input/pytorch-tcn-library/pytorch_tcn-1.2.3/'
    sys.path.append(tcn_source_path)
    
else:
    sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), "..", "..", "..")))

from pytorch_tcn import TCN
from src.utils.utils import get_models_save_path, get_submission_csv_path, running_in_kaggle, get_raw_data_dir
from src.utils.constants import Constants
from src.datasets.eeg_dataset import EEGDataset
from src.models.tcn import TCNModel 



2025-10-22 12:23:29,226 :: root :: INFO :: Initialising Utils
2025-10-22 12:23:29,979 :: root :: INFO :: Initialising Datasets
2025-10-22 12:23:30,006 :: root :: INFO :: Initialising Models


In [2]:
DATA_PATH = get_raw_data_dir()

class InfCFG:
    model_name = 'TCN'
    num_tcn_channels = [64, 128, 128, 256, 256, 512, 512, 512]
    kernel_size = 21 
    dropout = 0.35
    
    target_size = 6
    num_channels = 20 
    
    data_path = DATA_PATH
    n_splits = 5
    
    batch_size = 32
    num_workers = 0
    
    downsample_factor = 3
    
    if running_in_kaggle():
        MODEL_DIR = f'/kaggle/input/tcn-sum-votes/' 
    else:
        MODEL_DIR = get_models_save_path() / "TCNModel" / DATA_PREPARATION_VOTE_METHOD

InfCFG.model_paths = [os.path.join(InfCFG.MODEL_DIR, f'best_model_fold{i}.pth') for i in range(InfCFG.n_splits)]

TARGETS = Constants.TARGETS


In [3]:

def run_inference():
    """Executes the main inference loop."""
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"Using device: {device}")

    test_df = pd.read_csv(os.path.join(InfCFG.data_path, 'test.csv'))
    
    test_dataset = EEGDataset(df=test_df, data_path=InfCFG.data_path, mode='test', downsample_factor=InfCFG.downsample_factor)
    test_loader = DataLoader(
        test_dataset, 
        batch_size=InfCFG.batch_size, 
        shuffle=False, 
        num_workers=InfCFG.num_workers
    )

    all_fold_predictions = []

    for i, path in enumerate(InfCFG.model_paths):
        print(f"\n========== Inferencing with Fold {i} Model ==========")
        if not os.path.exists(path):
            print(f"Model file not found: {path}. Skipping this fold.")
            continue
            
        model = TCNModel(
            num_inputs=InfCFG.num_channels,
            num_outputs=InfCFG.target_size,
            channel_sizes=InfCFG.num_tcn_channels,
            kernel_size=InfCFG.kernel_size,
            dropout=InfCFG.dropout,
            causal=False,
            use_skip_connections=True
            
        )
        
        model.load_state_dict(torch.load(path, map_location=device))
        model.to(device)
        model.eval()

        current_fold_preds = []
        with torch.no_grad():
            for signals in tqdm(test_loader, desc=f"Predicting Fold {i}"):
                outputs = model(signals.to(device))
                probs = F.softmax(outputs, dim=1).cpu().numpy()
                current_fold_preds.append(probs)
        
        all_fold_predictions.append(np.concatenate(current_fold_preds))

    if not all_fold_predictions:
        print("No models were found for inference. Aborting.")
        return

    avg_predictions = np.mean(all_fold_predictions, axis=0)
    
    submission = pd.DataFrame({"eeg_id": test_df["eeg_id"]})
    submission[TARGETS] = avg_predictions
    submission.to_csv(get_submission_csv_path(), index=False)
    
    print("\nSubmission file created successfully!")
    print(submission.head())


In [4]:

if __name__ == '__main__':
    run_inference()

Using device: cuda

========== Inferencing with Fold 0 Model ==========


Predicting Fold 0:   0%|          | 0/1 [00:00<?, ?it/s]


========== Inferencing with Fold 1 Model ==========


Predicting Fold 1:   0%|          | 0/1 [00:00<?, ?it/s]


========== Inferencing with Fold 2 Model ==========


...


========== Inferencing with Fold 3 Model ==========


Predicting Fold 3:   0%|          | 0/1 [00:00<?, ?it/s]


========== Inferencing with Fold 4 Model ==========


Predicting Fold 4:   0%|          | 0/1 [00:00<?, ?it/s]


Submission file created successfully!
       eeg_id  seizure_vote  lpd_vote  gpd_vote  lrda_vote  grda_vote  \
0  3911565283      0.042847  0.064649  0.003721   0.067294   0.036041   

   other_vote  
0    0.785448  
